In [ ]:
!git pull

In [7]:
!git add .
!git commit -m "updated update script to not put redundant entries in db anymore"
!git push


The file will have its original line endings in your working directory
[master 3a69ed2f] updated update script to not put redundant entries in db anymore
 4 files changed, 19 insertions(+), 15 deletions(-)
To https://github.com/thekhan314/CrudeOilApp.git
   bbbdedb8..3a69ed2f  master -> master


In [1]:
from data_functions import *
import math
import datetime as dt
import sqlite3
import pandas as pd

from datashop import *

path = 'c:/prompt_root/CrudeOilApp'
relpath = '/data/nyt_api/nyt_jsons/'

conn = sqlite3.connect('data/energydash.db')

eia_dict = {
    'PET.WTTSTUS1.W':['WeeklyStocks','235081','%Y%m%d'],
    'PET.RWTC.D':['DailyPrice','241335','%Y%m%d'],
    'PET.WRPUPUS2.W':['ProductSupplied','401676','%Y%m%d'],
    'PET.E_ERTRRO_XR0_NUS_C.M':['MonthlyRigCount','296749','%Y%m']
}

#test_bulk = jsons_to_frame(path,relpath,conn)
#test_bulk.to_csv('data/nyt_api/nyt_articles.csv')

In [3]:
query = " ALTER TABLE DIA RENAME TO DIA_closing"
last_archive_date = conn.execute(query)


In [ ]:
# Lates News

query = " SELECT MAX(Date) FROM news LIMIT 1"
last_archive_date = str(conn.execute(query).fetchall()[0][0])

end_date = str(dtime.datetime.today().date())

query = 'Oil (Petroleum) and Gasoline'

recents_call = nytResp(last_archive_date,end_date,query)

if recents_call.hits > 0:

    df_hits = recents_call.frame

    query_lastnews = " SELECT * FROM news ORDER BY Date DESC LIMIT 50"
    df_lastnews = pd.read_sql(query_lastnews,conn)
    df_lastnews = df_lastnews[['Date','abstract''id']]

    df_hits = df_hits[~df_hits['id'].isin(df_lastnews['id'])]
    
    
    hits = len(df_hits)   

    if hits > 0:               
        
        records = df_hits[['abstract','Date','url']].to_dict(orient='records')
        for record in records:
            log_dict['News'].append(record)

        df_hits.to_sql('news',conn,if_exists='append')

In [ ]:
query_lastnews = " SELECT * FROM news WHERE Date < '2020-10-23 17:54:49.445231' ORDER BY Date DESC LIMIT 50"
df_lastnews = pd.read_sql(query_lastnews,conn)
df_lastnews = df_lastnews[['Date','abstract','WeeklyStocks','DIA_closing','DailyPrice','ProductSupplied','id']]
print(df_lastnews)

In [ ]:

last_archive_date = "2020-10-01 17:54:49.445231"

end_date = dt.datetime.today().date()

query = 'Oil (Petroleum) and Gasoline'

recents_call = nytResp(last_archive_date,end_date,query)

In [ ]:
df_recent = recents_call.frame
print(df_recent)


In [ ]:
df_new = df_recent[~df_recent['id'].isin(df_lastnews['id'])]

In [ ]:
print(df_new)

In [ ]:


query = " SELECT MAX(Date) FROM news LIMIT 1"
last_archive_date = conn.execute(query).fetchall()[0][0]
last_archive_date = "2020-10-23 17:54:49.445231"

end_date = dt.datetime.today().date()

query = 'Oil (Petroleum) and Gasoline'

recents_call = nytResp(last_archive_date,end_date,query)

query_lastnews = " SELECT * FROM news WHERE Date < 2020-11-05 17:54:49 ORDER BY Date DESC LIMIT 50"
df_lastnews = pd.read_sql(query_lastnews,conn)
df_lastnews = df[['Date','abstract','WeeklyStocks','DIA_closing','DailyPrice','ProductSupplied','id']]

display(df)

if recents_call.hits > 0:
    hits = len(recents_call.frame)                  
    df_current = pd.concat([df_current] * hits).reset_index()    
    
    df_news = pd.concat([
        recents_call.frame,
        df_current],
        axis=1)

    records = df_news[['abstract','Date','url']].to_dict(orient='records')
    for record in records:
        log_dict['News'].append(record)

    df_news.to_sql('news',conn,if_exists='append')

    #__________  record news updated time

cache_dict['news_update'] = end_date

In [4]:
# CHeck how many tables in db
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('MonthlyRigCount',), ('WeeklyStocks',), ('DailyPrice',), ('ProductSupplied',), ('DIA_closing',), ('news',)]


In [ ]:
# Last items in news
query = " SELECT * FROM news ORDER BY Date DESC LIMIT 50"
df = pd.read_sql(query,conn)
df = df[['Date','abstract','WeeklyStocks','DIA_closing','DailyPrice','ProductSupplied','id']]

display(df)

# Intra Day